In [5]:
import ccxt 
import telegram
import time
from datetime import datetime
import pandas as pd
import numpy as np
import talib
import math
from binance.client import Client
from binance_f import RequestClient

df_test = pd.read_pickle('SOLUSDT_2022_070809_3m.pkl')

In [6]:
# ma100
df_test['ma100'] = talib.MA(np.asarray(df_test['close']), 100)
df_test['ma100_incli'] = df_test.ma100 - df_test.ma100.shift()
df_test['ma100_degrees'] = df_test['ma100_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

# ma120
df_test['ma120'] = talib.MA(np.asarray(df_test['close']), 120)
df_test['ma120_incli'] = df_test.ma120 - df_test.ma120.shift()
df_test['ma120_degrees'] = df_test['ma120_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

# ma200
df_test['ma200'] = talib.MA(np.asarray(df_test['close']), 200)
df_test['ma200_incli'] = df_test.ma200 - df_test.ma200.shift()
df_test['ma200_degrees'] = df_test['ma200_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

# ma240
df_test['ma240'] = talib.MA(np.asarray(df_test['close']), 240)
df_test['ma240_incli'] = df_test.ma240 - df_test.ma240.shift()
df_test['ma240_degrees'] = df_test['ma240_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

# ma1200
df_test['ma1200'] = talib.MA(np.asarray(df_test['close']), 1200)
df_test['ma1200_incli'] = df_test.ma1200 - df_test.ma1200.shift()
df_test['ma1200_degrees'] = df_test['ma1200_incli'].apply(lambda x: math.degrees(math.atan2(x, 3*60))) 

macd, macdsignal, macdhist = talib.MACD(np.asarray(df_test['close']), 12, 26, 9)

df_test['macd'] = macd
df_test['signal'] = macdsignal
# df_test['macdhist'] = macdhist

# ma60 = df_test['ma60']
ma100 = df_test['ma100']
ma120 = df_test['ma120']
ma200 = df_test['ma200']
ma240 = df_test['ma240']
ma1200 = df_test['ma1200']

# ma60_deg_r = round(df_test['ma60_degrees'], 5)
ma100_deg_r = round(df_test['ma100_degrees'], 5)
ma120_deg_r = round(df_test['ma120_degrees'], 5)
ma200_deg_r = round(df_test['ma200_degrees'], 5)
ma240_deg_r = round(df_test['ma240_degrees'], 5)
ma1200_deg_r = round(df_test['ma1200_degrees'], 5)
macd_r = round(df_test['macd'], 5)
macdsignal_r = round(df_test['signal'], 5)

In [23]:
k = df_test[['datetime', 'close']]
take_porfit_l =0
stop_loss_l = 0
take_porfit_s =0
stop_loss_s = 0
position = 0 #포지션의 중복을 막는 변수
balance = 0 # 잔고
l_entry = 0
s_entry = 0
wait = 0 # 대기시간
mx = []

for i in range(len(df_test['close'])):
    if (wait != 0):
        wait = wait - 1

    if (wait==0) & (position==0) & (ma1200_deg_r[i]>0) & (df_test['close'][i]>ma1200[i]) & (df_test['close'][i]>ma100[i]) & (df_test['close'][i]>ma120[i]) & (df_test['close'][i]>ma200[i]) & (df_test['close'][i]>ma240[i]):
        position = 1
        print('==================LONG')
        print(k.iloc[i:i+1])
        l_entry = df_test['close'][i]
        take_porfit_l = round((df_test['close'][i] + df_test['close'][i]*0.03), 2)
        stop_loss_l = round((df_test['close'][i] - df_test['close'][i]*0.01), 2)
        print('TP :', take_porfit_l, 'SL :', stop_loss_l)
        # print('TP :', take_porfit_l)
        print('l_entry :',l_entry,' s_entry :', s_entry, ' l_po :', position)

    elif (wait==0) & (position==0) & (ma1200_deg_r[i]<0) & (df_test['close'][i]<ma1200[i]) & (df_test['close'][i]<ma100[i]) & (df_test['close'][i]<ma120[i]) & (df_test['close'][i]<ma200[i]) & (df_test['close'][i]<ma240[i]):
        position = 1
        print("=================SHORT")
        print(k.iloc[i:i+1])
        s_entry = df_test['close'][i]
        take_porfit_s = round((df_test['close'][i] - df_test['close'][i]*0.03), 2)
        stop_loss_s = round((df_test['close'][i] + df_test['close'][i]*0.01), 2)
        print('TP :', take_porfit_s, 'SL :', stop_loss_s)
        # print('TP :', take_porfit_s)
        print('l_entry :',l_entry,' s_entry :', s_entry, ' s_po :', position)
    
    # stop_loss_l = round(ma1200[i], 2)
    # stop_loss_s = round(ma1200[i], 2)
        
    # 롱 정리    
    if (df_test['high'][i] > take_porfit_l) & (position==1) & (l_entry!=0) & (s_entry==0):
        balance = balance + 2*(take_porfit_l - l_entry)
        mx.append(balance)
        position = 0
        take_porfit_l = 0
        stop_loss_l = 0
        l_entry = 0
        # wait = 60
        print('롱익절 : ', df_test['datetime'][i] , df_test['close'][i], '잔고 : ', round(balance, 2))
        
    elif (df_test['low'][i] < stop_loss_l) & (position==1) & (l_entry!=0) & (s_entry==0):
        balance = balance - 2*(l_entry - stop_loss_l)
        mx.append(balance)
        position = 0
        take_porfit_l = 0
        # print("스탑 : ", stop_loss_l)
        stop_loss_l = 0
        l_entry = 0
        wait = 60
        print('롱손절 : ', df_test['datetime'][i] ,df_test['close'][i], '잔고 : ', round(balance, 2))

    #숏 정리
    if (df_test['low'][i] < take_porfit_s) & (position==1) & (l_entry==0) & (s_entry!=0):
        balance = balance + 2*(s_entry -take_porfit_s)
        mx.append(balance)
        position = 0
        take_porfit_s = 0
        stop_loss_s = 0
        s_entry = 0
        wait = 60
        print('숏익절 : ', df_test['datetime'][i] ,df_test['close'][i], '잔고 : ', round(balance, 2))
         
    elif (df_test['high'][i] > stop_loss_s) & (position==1) & (l_entry==0) & (s_entry!=0):
        balance = balance - 2*(stop_loss_s - s_entry)
        mx.append(balance)
        position = 0
        take_porfit_s = 0
        # print("스탑 : ", stop_loss_s)
        stop_loss_s = 0
        s_entry = 0
        wait = 60
        print('숏손절 : ', df_test['datetime'][i] ,df_test['close'][i], '잔고 : ', round(balance, 2))
        

print('잔고 : ', round(balance, 2))
print("MAX : " ,round(max(mx), 2), "MIN : ",round(min(mx), 2))
print(mx)

=================SHORT
                 datetime  close
1214  2022-07-03 21:42:00  32.76
TP : 31.78 SL : 33.09
l_entry : 0  s_entry : 32.76  s_po : 1
숏손절 :  2022-07-04 04:30:00 33.11 잔고 :  -0.66
==================LONG
                 datetime  close
1410  2022-07-04 07:30:00  33.12
TP : 34.11 SL : 32.79
l_entry : 33.12  s_entry : 0  l_po : 1
롱손절 :  2022-07-04 11:45:00 32.78 잔고 :  -1.32
=================SHORT
                 datetime  close
1555  2022-07-04 14:45:00   32.8
TP : 31.82 SL : 33.13
l_entry : 0  s_entry : 32.8  s_po : 1
숏손절 :  2022-07-04 18:30:00 33.13 잔고 :  -1.98
==================LONG
                 datetime  close
1690  2022-07-04 21:30:00  34.25
TP : 35.28 SL : 33.91
l_entry : 34.25  s_entry : 0  l_po : 1
롱손절 :  2022-07-04 22:33:00 33.88 잔고 :  -2.66
==================LONG
                 datetime  close
1771  2022-07-05 01:33:00  34.21
TP : 35.24 SL : 33.87
l_entry : 34.21  s_entry : 0  l_po : 1
롱익절 :  2022-07-05 05:51:00 35.2 잔고 :  -0.6
==================LONG
     

100 : 7.59
80 : -0.67
60 : 10.28
40 : 8.63
20 : 6.08